In [ ]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt, dist
import geopy.distance
from scipy.stats import truncnorm

In [ ]:
#-----------------------------------EXTRACT THE DATA-----------------------------------

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
users_path = 'eua-dataset\\users\\'
U = pd.read_csv(users_path + 'users-test.csv')

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
servers_path = 'eua-dataset\\edge-servers\\'
S = pd.read_csv(servers_path + 'serverstest.csv')

In [ ]:
#-----------------------------------VARIABLES-----------------------------------

U_si = [] #set of individuals allocated to server si

D = (-37.81952,144.95714099999998)  #Position Danger
D_rad = 0.5  # influence range of danger source D (radius) in km

U_per = np.full(len(U),0.2)  # perception range of individual uj in km
R = np.zeros((4), dtype=int) # R = [CPU, RAM, VRAM, bandwidth]

# User u has memory requirements = w (example for 8 users)
w1 = [0.5, 0.5, 0.5, 2] 
w2 = [0.5, 1, 0.5, 2]
w3 = [1, 1, 0.5, 2]
w4 = [1, 1.5, 0.5, 2]
w5 = [1.5, 1.5, 1, 2]
w6 = [1, 1.5, 0.5, 2]
w7 = [0.5, 1, 0.5, 2]
w8 = [0.5, 0.5, 0.5, 2]
W = [w1, w2, w3, w4, w5, w6, w7, w8]   
  
# Each server has memory capacity c (example for 3 servers)
c1 = [2, 3, 1.5, 5]
c2 = [3, 4, 2.5, 4]
c3 = [5, 6, 2.5, 10]
C = [c1, c2, c3]    

T_1 =0 #Starting point of period
T_2 = 61 #Ending point of period
T = T_2-T_1 #Total interval time
ti = np.random.randint(T_1,T_2) #random time step in the given period

lambd = 0.5 #severity of the stimulus event
nej = np.random.uniform(0,1,len(U)) #emotional fluctuation of uj ---> nej ∈ (0, 1)
se_j =np.random.uniform(0.05,0.1, len(U)) #individual sensitivity uj 
    
du_Dt=[]  #distance between danger and user (change with t ?????????????????????)


In [ ]:
#-----------------------------------COVERAGE OF EACH SERVER-----------------------------------

def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

for i in range(len(S)):
  server_latitude = S.iloc[i]['LATITUDE']
  server_longitude = S.iloc[i]['LONGITUDE']
  temp = []

  for j in range(len(U)):
    user_latitude = U.iloc[j]['Latitude']
    user_longitude = U.iloc[j]['Longitude']

    radius = 0.03 # in kilometer

    dist_geo = haversine(server_longitude, server_latitude, user_longitude, user_latitude)
    #dist_temp.append(dist_geo)

    if dist_geo <= radius:
        temp.append(1)
        
    else:
        temp.append(0)

  U_si.append(temp)


In [ ]:
###Get user coordinates at time t ---------- RVO algorithm
""" def realTimeLocation():
    ti = np.random.randint(T_1,T_2)
    for j in range(len(U)):
        user_latitude = U.iloc[j]['Latitude']
        user_longitude = U.iloc[j]['Longitude']
    return ti,user_latitude,user_longitude """
    
for j in range(len(U)):
    user_latitude = U.iloc[j]['Latitude']
    user_longitude = U.iloc[j]['Longitude']
    user_coordinates = (user_latitude,user_longitude)
    dist_geoDanger = geopy.distance.geodesic(user_coordinates, D).km
    du_Dt.append(dist_geoDanger)

   

In [ ]:
#-----------------------------------CRITICALITY MODEL-----------------------------------

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def criticality (time):
    #emj: emotional value
    em_t = np.zeros(len(U))
    for j in range(len(U)):
        if du_Dt[j] < (D_rad+U_per[j]):
            eq = ((time-T_1)/T) * (1-(du_Dt[j]/(D_rad+U_per[j])))*lambd* nej[j]
            em_t[j]=eq
        else:
            em_t[j]=0

    # scrj(t) = sej + emj (t) (subjective criticality)
    scr=np.zeros(len(U))
    for j in range(len(U)):
        scr[j]= (se_j[j]+em_t[j])

    ocr_j=np.zeros(len(U)) #ocr objective criticality of individual uj at time step t
    nDiv = 5 #number of partitions of the coverage area
    circles = D_rad/nDiv #radius of each circular partition
    covCircles = [] #Distance from the center
    ri = [0.5,0.4,0.3,0.2,0.1] # it should be lambda * ri ????

    for p in range(1,nDiv+1):
        covCircles.append(circles*p)

    for j in range(len(U)):
            if du_Dt[j]<covCircles[0]:
                ocr_j[j] = ri[0]

            elif du_Dt[j]<covCircles[1] and du_Dt[j]>=covCircles[0]:
                ocr_j[j]  =ri[1]

            elif du_Dt[j]<covCircles[2] and du_Dt[j]>=covCircles[1]:
                ocr_j[j]  = ri[2]

            elif du_Dt[j]<covCircles[3] and du_Dt[j]>=covCircles[2]:
                ocr_j[j] = ri[3]
            elif du_Dt[j]<covCircles[4] and du_Dt[j]>=covCircles[3]:
                ocr_j[j]  = ri[4]

            else:
                ocr_j[j]  = 0


    # Criticality
    #CRj = (L1 * scrj(t)) + (L2 * ocrj)

    mu=1
    sigma=0.05
    low=0.95
    up =1.05

    X = get_truncated_normal(mu, sigma, low, up)

    #L1 =X.rvs() # weight lambda 1
    L1 = 0.95
    L2 = 1 # weight lambda 2

    #Criticality
    CR = []

    for j in range(len(U)):
        cr = (L1*scr[j])+(L2*ocr_j[j])
        CR.append(cr)

    index_CR=sorted(range(len(CR)), key=lambda a: CR[a],reverse=True)

    return index_CR, CR

In [ ]:
index_CR_t,CR_t =criticality(ti)

In [ ]:
#-----------------------------------MIP SOLVER-----------------------------------

solver = pywraplp.Solver.CreateSolver('SCIP')

In [ ]:
#-----------------------------------SOLVER VARIABLES-----------------------------------
# x_i,j = True if user u_j is allocated to edge server s_i
# x_i,j = False otherwise
x = {}
for j in index_CR_t:
    for i in range(len(S)):
        x[i, j] = solver.BoolVar(f'c[{i}][{j}]')

# y_i = True if Sum(xij) > 0, ∀i ∈ {1, . . . , m}
# y_i = False otherwise
y = {}
""" for i in range(len(S)):
                y[i] = solver.BoolVar(f'solver.Sum([x[i, j] for j in index_CR])>0') """

for i in range(len(S)):
    y[i] = solver.BoolVar(f'c[{i}]')              
                

print('Number of variables =', solver.NumVariables())

In [ ]:
#-----------------------------------CONSTRAINTS-----------------------------------

# Capacity constraint
for i in range(len(S)):
    for k in range(len(R)):
        solver.Add(
            solver.Sum([
                x[i,j]*W[j][k] for j in index_CR_t
            ]) <= C[i][k]*y[i]
        )
    
# Each user can be allocated just once
for j in index_CR_t:
    solver.Add(
        solver.Sum([x[i, j] for i in range(len(S))]) <= 1
    )

#Coverage
for j in index_CR_t:
    for i in range(len(S)):
        if U_si[i][j] == 0:
            solver.Add(
             x[i, j]==0
            )

#yi
""" for i in range(len(S)):
    if solver.Sum([x[i,j] for j in index_CR])==0:
        solver.Add(y[i]==0) """

print('Number of constraints =', solver.NumConstraints())

In [ ]:
#-----------------------------------OBJECTIVE FUNCTION-----------------------------------

# Maximize the number of allocated users
objective = solver.Objective()
for j in index_CR_t:
    for i in range(len(S)):
        objective.SetCoefficient(x[i, j], CR_t[j])
objective.SetMaximization()

# Minimize the number of required edge servers
""" objectiveMin = solver.Objective()

for i in S:
    objectiveMin.SetCoefficient(y[i], 1)
objectiveMin.SetMinimization() """

In [ ]:
#-----------------------------------CALL THE SOLVER-----------------------------------
status = solver.Solve()

#-----------------------------------DISPLAY THE SOLUTION-----------------------------------

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value x=', objective.Value())
    for j in index_CR_t:
        for i in range(len(S)):
            if int(x[i,j].solution_value()) == 1:
                print(f'User {j} has been allocated on server {i}')
else:
    print('The problem does not have an optimal solution.')